# Moist Static Energy Difference Analysis
This notebook evaluates whether the assumption that the difference between the surface and free troposphere MSE, $\epsilon = h - h_{FT}$ varies as the climate warms in the same way for the mean summer day as the temperature percentile $x$ day.

I.e. it evaluates to what extent the assumption of $\delta \epsilon_x = \delta \overline{\epsilon}$ is valid.

In [2]:
import sys
import os
# REMOTE - So can access functions in isca_tools which is in home/Isca directory
# sys.path.append(os.path.join(os.environ['HOME'], 'Isca'))
# LOCAL - So can access functions in isca_tools which is in StAndrews/Isca
sys.path.append(os.environ['PWD'])
import isca_tools
from isca_tools.utils.moist_physics import lcl_temp, rh_from_sphum, saturation_vapor_pressure, mixing_ratio_from_sphum, dry_profile, moist_profile, mixing_ratio_from_partial_pressure, moist_static_energy, convection_neutral_profile
from isca_tools.utils.radiation import frierson_net_toa_sw_dwn
from isca_tools.utils.constants import kappa, epsilon, L_v, c_p, g, R_v
from isca_tools.utils import area_weight_mean_lat, area_weighting
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import scipy.optimize
from tqdm import tqdm
import os

2023-03-24 17:43:05,576 - isca - WARNING - Environment variable GFDL_SOC not set, but this is only required if using SocratesCodebase. Setting to None


In [5]:
# Load dataset
exp_dir = 'tau_sweep/aquaplanet/'
var_keep = ['temp', 'sphum', 'height']      # only keep variables to compute MSE
pressure_ft = 500
use_time_start = 360*2
exp_names = [dir for dir in os.listdir(os.path.join(os.environ['GFDL_DATA'],exp_dir)) if dir[0]=='k']
exp_names.sort()
n_exp = len(exp_names)
ds = []
albedo = []
tau_sw = []
tau_lw = []
for i in tqdm(range(n_exp)):
    ds_use = isca_tools.load_dataset(exp_dir + exp_names[i])[var_keep]
    ds_use = ds_use.sel(time=slice(use_time_start, np.inf), drop=True)      # only use times after converged

    ds += [ds_use.sel(pfull=[pressure_ft, np.inf], method='nearest', drop=True).load()]     # only keep pressure values at surface and FT
    namelist = isca_tools.load_namelist(exp_dir + exp_names[i])  # Need this for albedo_value
    albedo += [namelist['mixed_layer_nml']['albedo_value']]
    tau_sw += [namelist['two_stream_gray_rad_nml']['atm_abs']]
    tau_lw += [namelist['two_stream_gray_rad_nml']['odp']]
pressure_ft_actual = float(ds[0].pfull[0])        # actual free troposphere pressure in hPa
pressure_surface = float(ds[0].pfull[1])          # surface pressure in hPa

100%|██████████| 8/8 [01:34<00:00, 11.81s/it]


## Get data
Get one dataset, `ds_all[i, j]` `tau_lw[i]` in latitude bin `j`, containing all summer data.

In [31]:
# Get data set averaged over all summer days in extratropics.
# Do this for each optical depth value
lat_min = [0, 40, 70]
lat_max = [20, 65, 90]
n_lat = len(lat_min)
n_pressure = ds[0].pfull.size
quant_plot = np.arange(100)     # first value must be 0 so temp_quant[0] is the mean value - make spacing 2 or something to speed up
n_quant = len(quant_plot)

temp_quant = np.zeros((n_exp, n_lat, n_quant, n_pressure))
sphum_quant = np.zeros((n_exp, n_lat, n_quant, n_pressure))
z_quant = np.zeros((n_exp, n_lat, n_quant, n_pressure))

with tqdm(total=n_exp * n_lat * n_quant, position=0, leave=True) as pbar:
    for i in range(n_exp):
        for j in range(n_lat):
            ds_use = ds[i].where((np.abs(ds[i].lat)<=lat_max[j]) & (np.abs(ds[i].lat)>=lat_min[j]), drop=True)

            # Only consider summer as has expected circulation
            ds_nh_summer = isca_tools.utils.annual_time_slice(ds_use, [7, 8, 9]).sel(lat=slice(lat_min, lat_max))       # NH summer is JAS
            ds_sh_summer = isca_tools.utils.annual_time_slice(ds_use, [1, 2, 3]).sel(lat=slice(-lat_max, -lat_min))     # SH summer is JFM
            # Combine hemispheres and average over longitude, time and latitude.
            # Note that April, May, June, October, November and December will not be included in this dataset
            ds_use = xr.concat([ds_sh_summer, ds_nh_summer], dim='lat')
            ds_use = ds_use.stack(lon_lat_time=("lat","lon","time"), create_index=False).chunk(dict(lon_lat_time=-1))
            # Get rid of nan values corresponding to the winter hemisphere - half the values are nan
            ds_use = ds_use.sel(lon_lat_time = ds_use.lon_lat_time[np.invert(np.isnan(ds_use.temp[-1]))].to_numpy())
            
            # Combine hemispheres and average over longitude, time and latitude.
            for k in range(n_quant):
                quant_thresh = ds_use.temp[:, -1].quantile(quant_plot[k]/100, dim=['time', 'lat', 'lon'])
                keep = ds_use.temp[:, -1] > quant_thresh
                keep = np.expand_dims(keep, 1).repeat(ds_use.temp.shape[1], axis=1)        # repeat keep for both pressure levels
                keep = xr.DataArray(keep, coords=ds_use.coords)                           # Make it a data array
                ds_use2 = ds_use.where(keep, drop=True)
                temp_quant[i, j, k] = ds_use2.temp.mean(dim=['time', 'lat', 'lon'])
                sphum_quant[i, j, k] = ds_use2.sphum.mean(dim=['time', 'lat', 'lon'])
                z_quant[i, j, k] = ds_use2.height.mean(dim=['time', 'lat', 'lon'])
                pbar.update(1)

  2%|▏         | 37/2400 [00:10<10:39,  3.69it/s]

KeyboardInterrupt

